# Warning

This tutorial is a work in progress, it does not cover all the functionalities yet. However, it will give you a good overview how you can generate tumor segmentations with BraTS Toolkit.


# glioma segmentation with BraTS Toolkit

In this Notebook, we will demonstrate how to preprocess brain MR images with the [BrainLes preprocessing package](https://github.com/BrainLesion/preprocessing/tree/main/brainles_preprocessing). Following we will generate glioma segmentations with multiple algorithms from [BraTS Toolkit](https://github.com/neuronflow/BraTS-Toolkit) and fuse them together.

---

## Getting Started

### Requirements

This tutorial requires:

- Python 3.10+ environment
- Docker (we recommended the most recent version)
<!-- TODO specify version -->

optional (but recommended):

- CUDA drivers
- a GPU that is supported (each algorithms supports different set of GPUs)
<!-- TODO specify -->

### Setup Colab environment

If you installed the packages and requirements on your own machine, you can skip this section and start from the import section. Otherwise you can follow and execute the tutorial on your browser. In order to start working on the notebook, click on the following button, this will open this page in the Colab environment and you will be able to execute the code on your own.

<a target="_blank" href="https://colab.research.google.com/github/BrainLesion/tutorials/blob/main/AURORA/tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Now that you are visualizing the notebook in Colab, run the next cell to install the packages we will use. There are few things you should follow in order to properly set the notebook up:

1. Warning: This notebook was not authored by Google. Click on 'Run anyway'.
1. When the installation commands are done, there might be "Restart runtime" button at the end of the output. Please, click it.


## Installation of dependencies


In [1]:
%pip install -U brainles_preprocessing brats_toolkit auxiliary
%pip install -U matplotlib

%load_ext autoreload
%autoreload 2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 89.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: brainles_preprocessing
    Found existing installation: brainles_preprocessing 0.0.25
    Uninstalling brainles_preprocessing-0.0.25:
      Successfully uninstalled brainles_preprocessing-0.0.25
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


By running the next cell you are going to create a folder in your Google Drive. All the files for this tutorial will be uploaded to this folder. After the first execution you might receive some warning and notification, please follow these instructions:

1. Permit this notebook to access your Google Drive files? Click on 'Yes', and select your account.
   Google Drive for desktop wants to access your Google Account. Click on 'Allow'.
1. At this point, a folder has been created and you can navigate it through the lefthand panel in Colab, you might also have received an email that informs you about the access on your Google Drive


In [6]:
# Create a folder in your Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
# Don't run this cell if you already cloned the repo
# !git clone https://github.com/BrainLesion/tutorials.git

In [2]:
# make files from the repo available in colab
import sys

COLAB_BASE_PATH = "/content/tutorials/BraTS-Toolkit/"
sys.path.insert(0, COLAB_BASE_PATH)

### Imports


## Raw Input data

TODO describe raw input data. Two exams from TCIA/TCGA with T1, T1c, T2, T2-FLAIR with skull etc.
TODO visualize it


## Preprocessing

BraTS challenge algorithms expect co-registered, skullstripped files in SRI-24 space, to achieve this preprocessing is required.
Instead of using the vanilla preprocessing pipeline from BraTS Toolkit, we recommend using the new [BrainLes preprocessing package](https://github.com/BrainLesion/preprocessing/tree/main/brainles_preprocessing).

<!-- For more details about preprocessing see the dedicated tutorial at TODO -->


First, we define a function that procoesses an exam.


In [ ]:
from auxiliary.normalization.percentile_normalizer import PercentileNormalizer
from auxiliary.turbopath import turbopath
from tqdm import tqdm

from brainles_preprocessing.brain_extraction import HDBetExtractor
from brainles_preprocessing.modality import Modality
from brainles_preprocessing.preprocessor import Preprocessor
from brainles_preprocessing.registration import NiftyRegRegistrator


def preprocess_exam_in_brats_style(inputDir: str) -> None:
    """
    Perform BRATS (Brain Tumor Segmentation) style preprocessing on MRI exam data.

    Args:
        inputDir (str): Path to the directory containing raw MRI files for an exam.

    Raises:
        Exception: If any error occurs during the preprocessing.

    Example:
        brat_style_preprocess_exam("/path/to/exam_directory")

    This function preprocesses MRI exam data following the BRATS style, which includes the following steps:
    1. Normalization using a percentile normalizer.
    2. Registration and correction using NiftyReg.
    3. Brain extraction using HDBet.

    The processed data is saved in a structured directory within the input directory.

    Args:
        inputDir (str): Path to the directory containing raw MRI files for an exam.

    Returns:
        None
    """
    inputDir = turbopath(inputDir)
    print("*** start ***")
    brainles_dir = turbopath(inputDir) + "/" + inputDir.name + "_brainles"
    norm_bet_dir = brainles_dir / "normalized_bet"
    raw_bet_dir = brainles_dir / "raw_bet"

    t1_file = inputDir.files("*t1.nii.gz")
    t1c_file = inputDir.files("*t1c.nii.gz")
    t2_file = inputDir.files("*t2.nii.gz")
    flair_file = inputDir.files("*fla.nii.gz")

    # we check that we have only one file of each type
    if len(t1_file) == len(t1c_file) == len(t2_file) == len(flair_file) == 1:
        t1File = t1_file[0]
        t1cFile = t1c_file[0]
        t2File = t2_file[0]
        flaFile = flair_file[0]

        # normalizer
        percentile_normalizer = PercentileNormalizer(
            lower_percentile=0.1,
            upper_percentile=99.9,
            lower_limit=0,
            upper_limit=1,
        )
        # define modalities
        center = Modality(
            modality_name="t1c",
            input_path=t1cFile,
            raw_bet_output_path=raw_bet_dir / inputDir.name + "_t1c_bet.nii.gz",
            normalized_bet_output_path=norm_bet_dir / inputDir.name
            + "_t1c_bet_normalized.nii.gz",
            atlas_correction=True,
            normalizer=percentile_normalizer,
        )
        moving_modalities = [
            Modality(
                modality_name="t1",
                input_path=t1File,
                raw_bet_output_path=raw_bet_dir / inputDir.name + "_t1_bet.nii.gz",
                normalized_bet_output_path=norm_bet_dir / inputDir.name
                + "_t1_bet_normalized.nii.gz",
                atlas_correction=True,
                normalizer=percentile_normalizer,
            ),
            Modality(
                modality_name="t2",
                input_path=t2File,
                raw_bet_output_path=raw_bet_dir / inputDir.name + "_t2_bet.nii.gz",
                normalized_bet_output_path=norm_bet_dir / inputDir.name
                + "_t2_bet_normalized.nii.gz",
                atlas_correction=True,
                normalizer=percentile_normalizer,
            ),
            Modality(
                modality_name="flair",
                input_path=flaFile,
                raw_bet_output_path=raw_bet_dir / inputDir.name + "_fla_bet.nii.gz",
                normalized_bet_output_path=norm_bet_dir / inputDir.name
                + "_fla_bet_normalized.nii.gz",
                atlas_correction=True,
                normalizer=percentile_normalizer,
            ),
        ]
        preprocessor = Preprocessor(
            center_modality=center,
            moving_modalities=moving_modalities,
            registrator=NiftyRegRegistrator(),
            brain_extractor=HDBetExtractor(),
            # optional: we provide a temporary directory as a sandbox for the preprocessin
            temp_folder="temporary_directory",
            limit_cuda_visible_devices="0",
        )
        preprocessor.run()

now we loop through the exams to preprocess the data:


In [ ]:
EXAMPLE_DATA_DIR = turbopath("data")

exams = EXAMPLE_DATA_DIR.dirs()

for exam in tqdm(exams):
    print("processing:", exam)
    preprocess_exam_in_brats_style(exam)

  0%|          | 0/2 [00:00<?, ?it/s]

processing: /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/data/OtherEXampleFromTCIA
*** start ***
File: /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/temporary_directory/atlas-space/atlas__t1c.nii.gz
preprocessing...
image shape after preprocessing:  (103, 160, 160)
prediction (CNN id)...
0
1
2
3
4
exporting segmentation...
current image /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/temporary_directory/brain-extraction/atlas_bet_t1c.nii.gz
current image /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/temporary_directory/brain-extraction/brain_masked/brain_masked__t1.nii.gz
current image /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/temporary_directory/brain-extraction/brain_masked/brain_masked__t2.nii.gz
current image /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/temporary_directory/brain-extraction/brain_masked/brain_masked__flair.nii.gz


 50%|█████     | 1/2 [03:51<03:51, 231.57s/it]

processing: /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/data/TCGA-DU-7294
*** start ***
File: /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/temporary_directory/atlas-space/atlas__t1c.nii.gz
preprocessing...
image shape after preprocessing:  (103, 160, 160)
prediction (CNN id)...
0
1
2
3
4
exporting segmentation...
current image /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/temporary_directory/brain-extraction/atlas_bet_t1c.nii.gz
current image /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/temporary_directory/brain-extraction/brain_masked/brain_masked__t1.nii.gz
current image /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/temporary_directory/brain-extraction/brain_masked/brain_masked__t2.nii.gz
current image /home/florian/flow/BrainLesion/tutorials/BraTS-Toolkit/temporary_directory/brain-extraction/brain_masked/brain_masked__flair.nii.gz


100%|██████████| 2/2 [11:31<00:00, 345.59s/it]


---


## segmentation with BraTS Toolkit

now that we have preprocessed the data we can start segmenting it with BraTS Toolkit.
Therefore, we again define a function to segment our files:


In [10]:
from brats_toolkit.segmentor import Segmentor
import os
from auxiliary.turbopath import turbopath
from tqdm import tqdm


def segment_exam(
    t1_file: str,
    t1c_file: str,
    t2_file: str,
    fla_file: str,
    segmentation_file: str,
    cid: str,
    cuda_device="0",
) -> None:
    """
    segment_exam - Segments MRI images using a specified algorithm from the BRATS toolkit.

    Parameters:
        t1_file (str): Path to the T1-weighted MRI image file.
        t1c_file (str): Path to the T1-weighted contrast-enhanced MRI image file.
        t2_file (str): Path to the T2-weighted MRI image file.
        fla_file (str): Path to the Fluid-attenuated inversion recovery (FLAIR) MRI image file.
        segmentation_file (str): Path to the output file where the segmented image will be saved.
        cid (str): Algorithm identifier for segmentation.
        cuda_device (str, optional): CUDA device ID for GPU acceleration. Default is "0".

    Returns:
        None

    Example:
        segment_exam(
            t1_file='/path/to/t1.nii.gz',
            t1c_file='/path/to/t1c.nii.gz',
            t2_file='/path/to/t2.nii.gz',
            fla_file='/path/to/fla.nii.gz',
            segmentation_file='/path/to/segmentation_result.nii.gz',
            cid="mic-dkfz"
        )

    This function segments MRI images using the specified algorithm from the BRATS toolkit. It accepts paths to T1-weighted,
    T1-weighted contrast-enhanced, T2-weighted, and FLAIR MRI images, and performs segmentation using the specified algorithm.
    The segmented image is saved in the specified output file.

    Note:
        - The function uses the BRATS toolkit's Segmentor for segmentation.
        - The 'cid' parameter specifies the algorithm to use.
        - Segmentation results are saved with a file name corresponding to the algorithm identifier (cid).
        - Errors during segmentation are caught, and an error message is printed.
    """

    # instantiate
    seg = Segmentor(
        verbose=True,
        gpu=cuda_device,
    )

    # algorithms we want to select for segmentation

    # execute it
    if not os.path.exists(segmentation_file):
        seg.segment(
            t1=t1_file,
            t2=t2_file,
            t1c=t1c_file,
            fla=fla_file,
            cid=cid,
            outputPath=segmentation_file,
        )
    else:
        print(segmentation_file, "already exists")

now we again loop through our exams:


In [19]:
EXAMPLE_DATA_DIR = turbopath("data")

exams = EXAMPLE_DATA_DIR.dirs()
# we use these five algorithms to segment the images
# 2019 algorithms
# cids = ["mic-dkfz", "scan", "xfeng", "lfb_rwth", "zyx_2019", "scan_2019"]

# 2020 algorithms
cids = ["isen-20", "hnfnetv1-20", "yixinmpl-20", "sanet0-20", "scan-20"]


for exam in tqdm(exams, desc="Exams"):
    for cid in tqdm(cids, desc="Algorithms", leave=False):
        print("segmenting:", exam, "with algorithm:", cid)
        brainles_folder = exam / exam.name + "_brainles"
        preprocessed_folder = brainles_folder / "raw_bet"
        segmentation_folder = brainles_folder / "segmentation"

        segmentation_file = segmentation_folder / cid + ".nii.gz"

        segment_exam(
            t1_file=preprocessed_folder / exam.name + "_t1_bet.nii.gz",
            t1c_file=preprocessed_folder / exam.name + "_t1c_bet.nii.gz",
            t2_file=preprocessed_folder / exam.name + "_t2_bet.nii.gz",
            fla_file=preprocessed_folder / exam.name + "_fla_bet.nii.gz",
            segmentation_file=segmentation_file,
            cid=cid,
            cuda_device="1",
        ),

Exams:   0%|          | 0/2 [00:00<?, ?it/s]

segmenting: /home/koflerf/tutorials/BraTS-Toolkit/data/OtherEXampleFromTCIA with algorithm: isen-20


──────────────────────────────────────── Thank you for using BraTS Toolkit ────────────────────────────────────────

Please support our development by citing BraTS Toolkit and the papers of the segmentation algorithms you use:

https://github.com/neuronflow/BraTS-Toolkit#citation -- Thank you!

...

Please support our development by citing BraTS Toolkit and the papers of the segmentation algorithms you use:

https://github.com/neuronflow/BraTS-Toolkit#citation -- Thank you!

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Exams: 100%|██████████| 2/2 [00:00<00:00, 11.06it/s]

/home/koflerf/tutorials/BraTS-Toolkit/data/TCGA-DU-7294/TCGA-DU-7294_brainles/segmentation/scan-20.nii.gz already exists


## Fusion

Now we can fuse the generated segmentations together. Therefore we again define a function to process our exams:


In [20]:
from brats_toolkit.fusionator import Fusionator


def fuse_segmentation_exam(
    segmentation_folder: str,
    fusion_folder: str,
):
    """
    Fuse multiple segmentations using different methods and save the results.

    Parameters:
    - segmentation_folder (str): Path to the folder containing segmentation files in NIfTI format (*.nii.gz).
    - fusion_folder (str): Path to the folder where fused segmentation results will be saved.

    Returns:
    None

    Example:
    ```python
    fuse_segmentation_exam("/path/to/segmentations", "/path/to/fusion_results")
    ```

    This function uses the Fusionator class from the brats_toolkit to fuse segmentations using two different methods:
    1. Multimodal Average (MAV) method.
    2. Simple fusion method.

    The fused segmentations are saved in the specified `fusion_folder` with file names "mav.nii.gz" and "simple.nii.gz".

    Note:
    - The Fusionator class must be available and correctly imported from the brats_toolkit.
    - The segmentation files in the `segmentation_folder` should be in NIfTI format (*.nii.gz).

    Args:
    - segmentation_folder (str): Path to the folder containing segmentation files in NIfTI format (*.nii.gz).
    - fusion_folder (str): Path to the folder where fused segmentation results will be saved.
    """
    # instantiate
    fus = Fusionator(verbose=True)

    # segmentation file paths
    segmentations = segmentation_folder.files("*.nii.gz")

    # execution
    # mav
    mavPath = fusion_folder / "mav.nii.gz"
    fus.fuse(
        segmentations=segmentations,
        outputPath=mavPath,
        method="mav",
        weights=None,
    )

    # simple
    simplePath = fusion_folder / "simple.nii.gz"
    fus.fuse(
        segmentations=segmentations,
        outputPath=simplePath,
        method="simple",
        weights=None,
    )

Now we can again loop through our exams:


In [21]:
EXAMPLE_DATA_DIR = turbopath("data")

exams = EXAMPLE_DATA_DIR.dirs()

for exam in tqdm(exams):
    print("segmenting:", exam)
    brainles_folder = exam / exam.name + "_brainles"
    segmentation_folder = brainles_folder / "segmentation"
    fusion_folder = brainles_folder / "fusion"

    fuse_segmentation_exam(
        segmentation_folder=segmentation_folder,
        fusion_folder=fusion_folder,
    )

  0%|          | 0/2 [00:00<?, ?it/s]

segmenting: /home/koflerf/tutorials/BraTS-Toolkit/data/OtherEXampleFromTCIA


──────────────────────────────────────── Thank you for using BraTS Toolkit ────────────────────────────────────────

Please support our development by citing BraTS Toolkit and the papers of the segmentation algorithms you use:

https://github.com/neuronflow/BraTS-Toolkit#citation -- Thank you!

...

Candidate with shape (155, 240, 240) and values [0] and sum 0
weight is: 1.0
weight is: 1
weight is: 1
weight is: 1.0


The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels other than 1 and 0.
The passed segmentation contains labels 

weight is: 1
Shape of result: (155, 240, 240)
Shape of current input array: (155, 240, 240)
Labels and datatype of current output: 0.0 0.0 float64
Convergence for label 4 after 0 iterations reached.
Shape of result: (155, 240, 240)
Shape of current input array: (155, 240, 240)
Labels and datatype of current output: 2.0 0.0 float64


Segmentation Fusion with method simple saved as /home/koflerf/tutorials/BraTS-Toolkit/data/TCGA-DU-7294/TCGA-DU-7294_brainles/fusion/simple.nii.gz.
Segmentation Fusion with method simple saved as /home/koflerf/tutorials/BraTS-Toolkit/data/TCGA-DU-7294/TCGA-DU-7294_brainles/fusion/simple.nii.gz.
Segmentation Fusion with method simple saved as /home/koflerf/tutorials/BraTS-Toolkit/data/TCGA-DU-7294/TCGA-DU-7294_brainles/fusion/simple.nii.gz.
Segmentation Fusion with method simple saved as /home/koflerf/tutorials/BraTS-Toolkit/data/TCGA-DU-7294/TCGA-DU-7294_brainles/fusion/simple.nii.gz.
Segmentation Fusion with method simple saved as /home/koflerf/tutorials/BraTS-Toolkit/data/TCGA-DU-7294/TCGA-DU-7294_brainles/fusion/simple.nii.gz.
Segmentation Fusion with method simple saved as /home/koflerf/tutorials/BraTS-Toolkit/data/TCGA-DU-7294/TCGA-DU-7294_brainles/fusion/simple.nii.gz.
Segmentation Fusion with method simple saved as /home/koflerf/tutorials/BraTS-Toolkit/data/TCGA-DU-7294/TCGA-DU-

Converting float64 to uint8 np.ndarray


<!-- ## Viz TODO add such visualizations## Data

AURORA expects _preprocessed_ input data as NIfTI file or NumPy Array (_preprocessed_ meaning the files should be co-registerend, skullstripped and in SRI-24 space).

In this example we provide sample data from the [ASNR-MICCAI BraTS Brain Metastasis Challenge](https://www.synapse.org/#!Synapse:syn51156910/wiki/622553), which is already preprocessed in the `AURORA/data` folder in the form of 4 modalities of the same brain (T1, T1C, T2, FLAIR). To get an intuition of the data, one example slice of the 3D scans is visualized below.

For your own data:
If the data is _not_ preprocessed yet, consider using our [BrainLes preprocessing](https://github.com/BrainLesion/preprocessing) package (or its predecessor [BraTS-Toolkit](https://github.com/neuronflow/BraTS-Toolkit)). -->


In [17]:
# utils.visualize_data()

<!-- ## Visualize results -->


In [ ]:
# utils.visualize_segmentation(
#     modality_file="data/t1c.nii.gz",
#     segmentation_file="output/t1c_segmentation.nii.gz",
# )